In [1]:
import tensorflow as tf
from tensorflow.keras import datasets
## [60k,28,28]
(x,y),(x_test,y_test) = datasets.mnist.load_data() ##下载数据集

x = tf.convert_to_tensor(x,dtype=tf.float32) / 255  #归一化

y = tf.convert_to_tensor(y,dtype=tf.int32)

x_test = tf.convert_to_tensor(x_test,dtype=tf.float32) / 255  #归一化

y_test = tf.convert_to_tensor(y_test,dtype=tf.int32)
# print(x.shape,y.shape,x.dtype,y.dtype)
# print(x_test.shape,y_test.shape,x_test.dtype,y_test.dtype)
#取数据集
train_db = tf.data.Dataset.from_tensor_slices((x,y)).batch(128)  ## 每次从中取出(128,28,28)个train数据
test_db = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(128)  ## 每次从中取出(128,28,28)个test数据
# train_iter = iter(train_db) # 迭代器
# sample = next(train_iter) 
# print("参数：",sample[0].shape,sample[1].shape)
# print(train_db)
#定义参数

""""
x [b,28,28]
y [b]
[b,784] => [b,256] => [b,128] =>[b,10]
w1 [784,256]  w2 [256,128] w3 [128,10]
b1 [256]       b2 [128]     b3 [10]
"""
w1 = tf.Variable(tf.random.truncated_normal([784,256],mean=0,stddev=0.1))
b1 = tf.Variable(tf.zeros(256))
w2 = tf.Variable(tf.random.truncated_normal([256,128],mean=0,stddev=0.1))
b2 = tf.Variable(tf.zeros(128))
w3 = tf.Variable(tf.random.truncated_normal([128,10],mean=0,stddev=0.1))
b3 = tf.Variable(tf.zeros(10))

# print(w1.shape,b1.shape)
# w1.numpy()
# print(b1.shape)




In [2]:

##设置相关参数
lr = 1e-3  ## 学习率 learning rate  
total_correct , total_num = 0 , 0

In [3]:
for iters in range(100): #迭代多次，使技术更加好用的一个方法是什么？
    for step,(x,y) in enumerate(train_db):
        with tf.GradientTape() as tape: ## 梯度下降函数
            x = tf.reshape(x,[-1,784]) # 转换格式
            # print(x.shape)
            h1 = tf.nn.relu(x@w1+b1) 
            h2 = tf.nn.relu(h1@w2+b2)
            out = h2@w3+b3
            ## compute loss
            ## 1. one hot for y
            y_onehot = tf.one_hot(y,depth=10)
            loss = tf.square(y_onehot - out)
            loss = tf.reduce_mean(loss) #计算误差的均值
            #梯度下降 更新权值
            grades = tape.gradient(loss,[w1,b1,w2,b2,w3,b3])
            w1.assign_sub(lr*grades[0])
            b1.assign_sub(lr*grades[1])
            w2.assign_sub(lr*grades[2])
            b2.assign_sub(lr*grades[3])
            w3.assign_sub(lr*grades[4])
            b3.assign_sub(lr*grades[5])
        if step % 100 == 0:
            print(iters,step,"loss",float(loss))
        
    ## 添加数据测试
    for step,(x_test,y_test) in enumerate(test_db):
        x_t = tf.reshape(x_test,[-1,784]) # 转换格式
        h1 = tf.nn.relu(x_t@w1+b1) 
        h2 = tf.nn.relu(h1@w2+b2)
        out = h2@w3+b3
        prob = tf.nn.softmax(out,axis = 1) ## 概率归一化
        pred = tf.cast(tf.argmax(prob,axis = 1),tf.int32)
        correct = tf.cast(tf.equal(pred,y_test),dtype = tf.int32)
        total_correct_sum = tf.reduce_sum(correct)
        # print(total_sum.numpy())
        total_correct += total_correct_sum ## 实时预测准确率
        total_num += y_test.shape[0]  ## 预测数据
        # print(y_test.numpy())      
    acc = total_correct/total_num
    print("acc:",acc)

0 0 loss 0.2559637427330017
0 100 loss 0.19778630137443542
0 200 loss 0.17760102450847626
0 300 loss 0.1733098328113556
0 400 loss 0.1663389652967453
acc: tf.Tensor(0.1129, shape=(), dtype=float64)
1 0 loss 0.14883890748023987
1 100 loss 0.15440988540649414
1 200 loss 0.14393657445907593
1 300 loss 0.14405062794685364
1 400 loss 0.13942301273345947
acc: tf.Tensor(0.1435, shape=(), dtype=float64)
2 0 loss 0.12561002373695374
2 100 loss 0.13369126617908478
2 200 loss 0.12518736720085144
2 300 loss 0.1261242926120758
2 400 loss 0.12311599403619766
acc: tf.Tensor(0.1767, shape=(), dtype=float64)
3 0 loss 0.11123672872781754
3 100 loss 0.12030700594186783
3 200 loss 0.11308082193136215
3 300 loss 0.11411155760288239
3 400 loss 0.11211265623569489
acc: tf.Tensor(0.2088, shape=(), dtype=float64)
4 0 loss 0.10141037404537201
4 100 loss 0.11088179051876068
4 200 loss 0.1046125739812851
4 300 loss 0.1054578572511673
4 400 loss 0.10415296256542206
acc: tf.Tensor(0.23842, shape=(), dtype=float64)


acc: tf.Tensor(0.5793076923076923, shape=(), dtype=float64)
39 0 loss 0.04985794797539711
39 100 loss 0.056328725069761276
39 200 loss 0.052995823323726654
39 300 loss 0.05375336483120918
39 400 loss 0.055615752935409546
acc: tf.Tensor(0.5832225, shape=(), dtype=float64)
40 0 loss 0.049475472420454025
40 100 loss 0.05589022487401962
40 200 loss 0.05256810039281845
40 300 loss 0.05335993692278862
40 400 loss 0.05524333566427231
acc: tf.Tensor(0.5870170731707317, shape=(), dtype=float64)
41 0 loss 0.0491054430603981
41 100 loss 0.05546564981341362
41 200 loss 0.05215432494878769
41 300 loss 0.052979934960603714
41 400 loss 0.05488339811563492
acc: tf.Tensor(0.5907261904761905, shape=(), dtype=float64)
42 0 loss 0.04874834045767784
42 100 loss 0.05505496263504028
42 200 loss 0.05175727605819702
42 300 loss 0.05261043459177017
42 400 loss 0.05453328415751457
acc: tf.Tensor(0.5943348837209302, shape=(), dtype=float64)
43 0 loss 0.04840311408042908
43 100 loss 0.054654717445373535
43 200 los

76 200 loss 0.043015073984861374
76 300 loss 0.044225551187992096
76 400 loss 0.04640202969312668
acc: tf.Tensor(0.6790909090909091, shape=(), dtype=float64)
77 0 loss 0.040612973272800446
77 100 loss 0.04555554687976837
77 200 loss 0.04283710569143295
77 300 loss 0.04405730962753296
77 400 loss 0.046228956431150436
acc: tf.Tensor(0.6808205128205128, shape=(), dtype=float64)
78 0 loss 0.04045488312840462
78 100 loss 0.04536407068371773
78 200 loss 0.042662613093853
78 300 loss 0.04389242082834244
78 400 loss 0.046058736741542816
acc: tf.Tensor(0.6825291139240506, shape=(), dtype=float64)
79 0 loss 0.040299344807863235
79 100 loss 0.04517628997564316
79 200 loss 0.04249114543199539
79 300 loss 0.04373084381222725
79 400 loss 0.04589097946882248
acc: tf.Tensor(0.68420875, shape=(), dtype=float64)
80 0 loss 0.040146030485630035
80 100 loss 0.04499176889657974
80 200 loss 0.042322415858507156
80 300 loss 0.04357265681028366
80 400 loss 0.045726001262664795
acc: tf.Tensor(0.6858493827160493